In [2]:

from sklearn.metrics import mean_squared_error
 
 
print("User Current Version:-", sys.version)

User Current Version:- 3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)]


In [3]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from collections import Counter
from IPython.core.display import display, HTML
from sklearn.linear_model import Ridge

C:\Users\Daniel\AppData\Local\Temp\ipykernel_5604\2007514277.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd

df = pd.read_excel('data.xlsx')

In [5]:
X = df.iloc[:, 0:19].values
y = df.iloc[:, 19].values.reshape(-1,1)

df.shape, X.shape, type(X)

((4665, 20), (4665, 19), numpy.ndarray)

In [25]:
from sklearn.model_selection import train_test_split
# X = X[:,cols]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 25)


# Fitting regressior to the Training set
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from numpy import savetxt ,loadtxt

steps = [
    ('scalar', StandardScaler()),
    ('feature_slection',SelectKBest(mutual_info_regression,k=10)),
    ('scalar2', StandardScaler()),
    ('poly', PolynomialFeatures(degree=4)),
    ('scalar3', StandardScaler()),
    ('model', ElasticNet(alpha=0.5,l1_ratio=0.6))
]

ElasticNet_pipe = Pipeline(steps)

X_test = loadtxt('X_test.csv', delimiter=',')
X_train = loadtxt('X_train.csv', delimiter=',')
y_test = loadtxt('y_test.csv', delimiter=',')
y_train = loadtxt('y_train.csv', delimiter=',')

In [13]:
ElasticNet_pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scalar', 'feature_slection', 'scalar2', 'poly', 'model', 'scalar__copy', 'scalar__with_mean', 'scalar__with_std', 'feature_slection__k', 'feature_slection__score_func', 'scalar2__copy', 'scalar2__with_mean', 'scalar2__with_std', 'poly__degree', 'poly__include_bias', 'poly__interaction_only', 'poly__order', 'model__alpha', 'model__copy_X', 'model__fit_intercept', 'model__l1_ratio', 'model__max_iter', 'model__normalize', 'model__positive', 'model__precompute', 'model__random_state', 'model__selection', 'model__tol', 'model__warm_start'])

In [26]:
from skopt import BayesSearchCV
from skopt.space import Real

from sklearn.metrics import fbeta_score, make_scorer
scoring_func = make_scorer(mean_squared_error)
bayes_search = BayesSearchCV(estimator = ElasticNet_pipe, 
                           search_spaces= {
                                'model__alpha':Real(1e-6, 1, prior='log-uniform'),
                                'model__l1_ratio':Real(1e-6, 1, prior='log-uniform')                          
                           },
                           scoring = scoring_func,
                           cv = 10,
                           n_jobs = -1);
bayes_search = bayes_search.fit(X_train, y_train);
bayes_search.score(X_test,y_test), bayes_search.cv_results_['mean_test_score'].mean()

C:\Users\Daniel\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.153e+06, tolerance: 1.621e+03
  model = cd_fast.enet_coordinate_descent(


(2272.4672457341258, 1926.0123846924405)

In [17]:

from sklearn.model_selection import GridSearchCV

parameters = [ {'model__alpha': np.logspace(-1, 1,num=3),'model__l1_ratio':np.arange(0.4,0.8,0.1) } ]

from sklearn.metrics import fbeta_score, make_scorer
scoring_func = make_scorer(mean_squared_error)
# grid = dict()
# grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
# grid['l1_ratio'] = np.arange(0.3, 1, 0.01)
grid_search = GridSearchCV(estimator = ElasticNet_pipe, 
                           param_grid = parameters,
                           scoring = scoring_func,
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)  
grid_search.score(X_test,y_test), grid_search.cv_results_['mean_test_score'].mean()

(2858.8668449384654, 2150.458635762094)

In [28]:
import pickle

def saveIfGood(model,X_test,y_test):
    # filenames = ['best_model_1.sav','best_model_2.sav','best_model.sav','best_model.sav','best_model.sav','best_model.sav','best_model.sav','best_model.sav']
    filename = 'best_model.sav'
    loaded_model = pickle.load(open(filename, 'rb'))
    y_pred = loaded_model.predict(X_test)
    y_pred_new = model.predict(X_test)
    mse_new = mean_squared_error(y_test, y_pred_new)
    mse_best = mean_squared_error(y_test, y_pred)
    print(loaded_model.get_params())
    print(loaded_model)

    if mse_new < mse_best:
        filename = 'best_model.sav'
        saveModel(model,filename)
        print(mse_new)
        print("Se guardo mejor")
    else:
        print("No es mejor:")
        print("mse_new:",mse_new)
        print("mse_best:",mse_best)

def saveModel(model,filename):
    pickle.dump(model, open(filename, 'wb'))

def ordenarModelos()


for i in range(len(filenames)):
    saveIfGood(filenames[i])


    
    
        

best_model = bayes_search.best_estimator_
saveIfGood(best_model,X_test,y_test)



{'memory': None, 'steps': [('scalar', StandardScaler()), ('feature_slection', SelectKBest(score_func=<function mutual_info_regression at 0x00000247574BCE00>)), ('poly', PolynomialFeatures(degree=3)), ('scalar2', StandardScaler()), ('model', Ridge(alpha=1.0028382472825777e-06))], 'verbose': False, 'scalar': StandardScaler(), 'feature_slection': SelectKBest(score_func=<function mutual_info_regression at 0x00000247574BCE00>), 'poly': PolynomialFeatures(degree=3), 'scalar2': StandardScaler(), 'model': Ridge(alpha=1.0028382472825777e-06), 'scalar__copy': True, 'scalar__with_mean': True, 'scalar__with_std': True, 'feature_slection__k': 10, 'feature_slection__score_func': <function mutual_info_regression at 0x00000247574BCE00>, 'poly__degree': 3, 'poly__include_bias': True, 'poly__interaction_only': False, 'poly__order': 'C', 'scalar2__copy': True, 'scalar2__with_mean': True, 'scalar2__with_std': True, 'model__alpha': 1.0028382472825777e-06, 'model__copy_X': True, 'model__fit_intercept': True

In [39]:
filename = 'best_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

y_pred = loaded_model.predict(X_test)
# y_pred_new = model.predict(X_test)
# mse_new = mean_squared_error(y_test, y_pred_new)
mse_best = mean_squared_error(y_test, y_pred)


'mod1769.sav'

In [42]:
ElasticNet_pipe.set_params('l1_ratio')


<bound method Pipeline.set_params of Pipeline(steps=[('scalar', StandardScaler()),
                ('feature_slection',
                 SelectKBest(score_func=<function mutual_info_regression at 0x00000247574BCE00>)),
                ('scalar2', StandardScaler()),
                ('poly', PolynomialFeatures(degree=4)),
                ('scalar3', StandardScaler()),
                ('model', ElasticNet(alpha=0.5, l1_ratio=0.6))])>

array([132.01056822, 134.25371139, 123.7840261 , ...,  77.31698818,
       149.75738482,   1.05178186])

In [59]:
y_pred[y_pred<0] = 0
mean_squared_error(y_test,y_pred)

2252.424293033005

In [50]:
y_pred = best_model.predict(X_test)
df_y_pred = pd.DataFrame (y_pred)
df_y_test = pd.DataFrame(y_test)
y_comp  
## save to xlsx file

filepath_pred = 'my_excel_pred.xlsx'
filepath = 'my_excel_file.xlsx'

df_y_pred.to_excel(filepath_pred, index=False)
df_y_test.to_excel(filepath, index=False)


In [63]:

np.count_nonzero(y_pred>0)
np.count_nonzero(y_test>=0)



best_model.get_params()

{'memory': None,
 'steps': [('scalar', StandardScaler()),
  ('feature_slection',
   SelectKBest(score_func=<function mutual_info_regression at 0x00000247574BCE00>)),
  ('scalar2', StandardScaler()),
  ('poly', PolynomialFeatures(degree=4)),
  ('scalar3', StandardScaler()),
  ('model', ElasticNet(l1_ratio=2.164263412821053e-05))],
 'verbose': False,
 'scalar': StandardScaler(),
 'feature_slection': SelectKBest(score_func=<function mutual_info_regression at 0x00000247574BCE00>),
 'scalar2': StandardScaler(),
 'poly': PolynomialFeatures(degree=4),
 'scalar3': StandardScaler(),
 'model': ElasticNet(l1_ratio=2.164263412821053e-05),
 'scalar__copy': True,
 'scalar__with_mean': True,
 'scalar__with_std': True,
 'feature_slection__k': 10,
 'feature_slection__score_func': <function sklearn.feature_selection._mutual_info.mutual_info_regression(X, y, *, discrete_features='auto', n_neighbors=3, copy=True, random_state=None)>,
 'scalar2__copy': True,
 'scalar2__with_mean': True,
 'scalar2__with_std